In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [6]:
df = pd.read_csv('datasets/train.csv', sep=',', encoding='CP949')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [23]:
# 분석에 사용할 변수 선택 : Pclass, Sex, Age
print('Pclass\n',df.Pclass.value_counts())
print('Sex\n',df.Sex.value_counts())
print('Age\n',df.Age.value_counts())

Pclass
 3    491
1    216
2    184
Name: Pclass, dtype: int64
Sex
 male      577
female    314
Name: Sex, dtype: int64
Age
 24.00    30
22.00    27
18.00    26
19.00    25
28.00    25
         ..
36.50     1
55.50     1
0.92      1
23.50     1
74.00     1
Name: Age, Length: 88, dtype: int64


In [24]:
# Age 함수 결측치 처리 - 평균값 대치
df1=df.copy()
df.Age.mean()
df1['Age'].fillna(df.Age.mean(), inplace=True)
df1.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [26]:
df2 = pd.get_dummies(df['Sex'], prefix='Sex', drop_first=False)
df3 = pd.concat([df1,df2],axis=1)

In [27]:
df3.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_female,Sex_male
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1


In [30]:
#종속변수와 독립변수 구분하기
df3['intersept']=1.0
x=df3[['intersept','Pclass','Age', 'Sex_male']]
y=df3[['Survived']]
#로지스틱 회귀분석 실행하기
logit=sm.Logit(y,x).fit()

Optimization terminated successfully.
         Current function value: 0.451923
         Iterations 6


In [31]:
#분석결과
print(logit.summary2())
print('\n오즈비:\n',np.exp(logit.params))
cf_m2=pd.DataFrame(logit.pred_table())
cf_m2.columns=['예측0','예측1']
cf_m2.index=['실제0','실제1']
print('\n분류행렬표:\n',cf_m2)

                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.321     
Dependent Variable: Survived         AIC:              813.3262  
Date:               2022-01-18 21:30 BIC:              832.4956  
No. Observations:   891              Log-Likelihood:   -402.66   
Df Model:           3                LL-Null:          -593.33   
Df Residuals:       887              LLR p-value:      2.4502e-82
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     6.0000                                       
------------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
------------------------------------------------------------------
intersept     4.7320    0.4498   10.5197  0.0000   3.8503   5.6136
Pclass       -1.1685    0.1189   -9.8239  0.0000  -1.4016  -0.9353
Age          -0.0334    0.0073   -4.5494  0.0000  -0.0478  -0.0190
Sex_male     -2.6120    0.1866

유사결정계수(Pseudo R-squared)는 0.321으로 유의성이 높지는 않은 모형으로 나타났다.  
절편의 값은 2.12이며 각 변수의 유의수준을 0.1 기준으로 하였을때,  
모든 변수는 유의하게 나타난다.  
오즈비 테이블에서 모든변수의 값이 1보다 작으므로, 변수의 값이 1단위가 커질때마다 생존확률이 점점 낮아진다.  
생존확률에 대한 영향력은 남성>Pclass>Age 순서로 부정적인 영향을 주는 것으로 나타났다.  
즉, 남성이고, Pclass가 낮을수록, Age가 높을수록 생존확률이 낮다.

In [8]:
df = pd.read_csv('datasets/train.csv', sep=',', encoding='CP949')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
# 결측치 채우기(평균값), 성별(문자열)을 숫자로 변환하기
df['Age'].fillna(df.Age.mean(), inplace=True)
df['Sex1']=df['Sex'].map({'female':0,'male':1})
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex1
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [10]:
# Pclass feature 변환하기
df['FirstClass'] = df['Pclass'].apply(lambda x: 1 if x == 1 else 0)
df['SecondClass'] = df['Pclass'].apply(lambda x: 1 if x == 2 else 0)
features = df[['Sex1', 'Age', 'FirstClass', 'SecondClass']]
survival = df['Survived']

In [11]:
#학습/평가세트 분리하기
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, survival)

In [13]:
#데이터 정규화(스케일링)
#로지스틱 회귀의 경우 regularation을 사용하기 때문에 그전에 반드시 스케일링이 필요
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

In [59]:
#모델생성
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train_features, train_labels)

LogisticRegression()

In [60]:
#학습세트 정확성 확인
print(model.score(train_features, train_labels))

0.7964071856287425


In [61]:
#평가세트 정확성 확인
print(model.score(test_features, test_labels))

0.8026905829596412


In [63]:
#feature들의 계수(coefficients)를 확인
print('Sex1', 'Age', 'FirstClass', 'SecondClass')
print(model.coef_)

Sex1 Age FirstClass SecondClass
[[-1.19962322 -0.45579997  0.95680174  0.47621625]]


성별이 1(남자)일 경우 생존에 불리하며(음수), 가장 영향력(절대값)이 크다.  
1등석일 경우 생존에 유리하며(양수)이며 2번째 영향력이다.  
나이가 많을수록 불리하며, 영향력은 2등석과 유사하다.

In [68]:
열공 = np.array([0.0, 45.0, 1.0, 0.0])
판다 = np.array([1.0, 30.0, 1.0, 0.0])
까치 = np.array([1.0, 38.0, 0.0, 1.0])
sample = np.array([열공, 판다, 까치])

In [69]:
sample = scaler.transform(sample)

In [70]:
print(model.predict(sample))
print(model.predict_proba(sample))

[1 0 0]
[[0.13345169 0.86654831]
 [0.52908796 0.47091204]
 [0.80254688 0.19745312]]


열공은 생존, 판다와 까치는 죽는걸로 나온다.  
열공은 생존가능성 86.7%, 판다는 생존가능성 47%, 까치는 19.7% 이다.